# COSA E' UN CHATBOT RAG E COME FUNZIONA
### Edizione PTMP 
##### (PocaTeoria MoltaPratica)

Un modello di intelligenza artificiale generativa ha una vasta conoscenza generale ereditata dalle fonti che sono state utilizzate per allenarlo.

Se vogliamo "insegnare" ad un modello generativo qualcosa che integri la sua conoscenza generale, possiamo seguire due strade:

- Fine Tuning : creiamo un dataset di Fine Tuning, e ri alleniamo il modello di GenAI su questi dati. Presenta pregi e difetti. Ne parliamo in un altro notebook.

- Retrieve and Generate : forniamo al modello le informazioni complementari direttamente durante l'inferenza. E' il paradigma che affronteremo in questo notebook.

## FASE 1 : "RETRIEVE"

In [ ]:
# importo dipendenze e variabili d'ambiente

import pandas as pd
import numpy as np
from ast import literal_eval
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

### Questa è la base di conoscenza di produzione di LIA.

In [ ]:
df = pd.read_csv("data\embeddings\embeddings_100.csv", index_col=0)         # carico il csv e lo rendo un dataframe
df['embeddings'] = df['embeddings'].apply(literal_eval).apply(np.array)     # passo gli embeddings da stringa a array
df                                                                          # visualizzo il dataframe

### Osserviamo come un "embedding" sia effettivamente un array di 1536 floats. 

In [ ]:
embs = [emb for emb in df.embeddings] 
print("Tipo:", type(embs[0]))
print("Lunghezza:", len(embs[0]))


### Ora teniamo soltanto le parti che ci servono: colonne title, text e embeddings.

In [ ]:
df = df[["title", "text", "embeddings"]]  

In [ ]:
df

### Facciamo ora una funzione che, con una chiamata a ada-002, calcola l'embedding di una frase di nostra scelta.

In [ ]:
# Creiamo un'istanza dell'SDK di openai. Lo considereremo un singleton in tutto il notebook.
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def genera_embeddings(testo:str) -> np.ndarray: 
    
    response = client.embeddings.create(input=testo,
                                        model='text-embedding-ada-002')
    return response.data[0].embedding

In [ ]:
nuovi_embeddings = genera_embeddings("Ciao, mi chiamo Lorenzo.")

print(nuovi_embeddings)

### Definiamo ora una funzione per la distanza coseno. 
PS: ricordiamo, per i più ruspanti, che la distanza coseno è il prodotto scalare / il prodotto delle norme.

Utilizziamo la distanza coseno perchè la magnitudine non ci interessa. Perchè non ci interessa? Perchè i modelli di embeddings restituiscono sempre array della stessa lunghezza. (nel caso di ada 1536).

Contrariamente a quanto potrebbe sembrare, migliore è il modello, minore è la lunghezza dei vettori che produce. I modelli precedenti ad ada-002 restituivano embeddings con lunghezze comprese tra 10.000 e 20.000 a seconda del modello, rendendo piu lunghe le elaborazioni. Si poteva ridurre la dimensionalità con la PCA, ma si perdeva MOLTA accuratezza. 

In [ ]:
def distanza_coseno(a:np.ndarray,b:np.ndarray)->float:
    
    return 1 - np.dot(a,b)/np.linalg.norm(a)*np.linalg.norm(b)

### Usiamo pandas per creare una colonna "distanza" che contiene la distanza_coseno tra l'embedding della funzione che noi gli diamo e tutte quelle nel dataframe

In [ ]:
b = genera_embeddings("avete una soluzione che sostituisce la filiale fisica?")

# creiamo la colonna distanza
df['distanza'] = df['embeddings'].apply(lambda a : distanza_coseno(a,b))

# ordiniamo il df per quella distanza 
df.sort_values('distanza')



### Ora stessa cosa, ma facciamo una funzione, che tiene solo la prima riga, e estrae text e title

In [ ]:
def esito_ricerca(frase:str) -> str:    
    
    # genero embeddings della mia frase, e valuterò le 
    b = genera_embeddings(frase)
    
    # creiamo la colonna distanza
    df['distanza'] = df['embeddings'].apply(lambda a : distanza_coseno(a,b))
    
    # prendiamo il primo elemento del df ordinato per la distanza
    riga_migliore = df.sort_values('distanza').iloc[0]
    
    # impacchettiamo il tutto in una stringa con forma "titolo : testo"
    risultato = f"{riga_migliore['title']} : {riga_migliore['text']}"
    
    return risultato

### Adesso siamo in grado, a partire da una frase, di estrarre il testo più pertinente dalla base di conoscenza e il suo titolo.

In [ ]:
esito_ricerca("come posso essere assunto?")

## Passiamo ora alla seconda parte: GENERATE

Ora passiamo alla parte vera e propria della generazione della risposta, ma parliamo prima un attimo dei modi in cui si può generare un testo generico utilizzando AI generativa, focalizzandoci non sulla teoria ma sulla pratica.

A prescinere dal modello usato, da chi lo ha realizzato, e dall'infrastruttura sistemistica che si occupa dell'inferenza, i metodi sono progettati per potere essere inferiti in due modi:

- con un PROMPT (modelli con nome xxx-instruct)
- con un CHAT COMPLETION (modelli con nome xxx-chat)

I modelli instruct sono sempre meno utilizzati, ma consentono di generare un testo a partire da un'istruzione in linguaggio naturale. Ecco un esempio:



In [ ]:
response = client.completions.create(prompt="Raccontami una storia divertente in italiano",
                                    model = "gpt-3.5-turbo-instruct",
                                    max_tokens=1000,
                                    temperature=1
                                    )

response.choices[0].text

I modelli di chat, ovvero quelli più potenti e moderni, seguono una sintassi di generazione del testo che invia una "chat" al sistema, il quale deve generare il completamento più logico a quella chat.

Viene inviata al modello una lista di messaggi, ciascun messaggio è un dizionario con chiavi "role" e "content".

I ruoli possono essere "system", "user" o "assistant" o "function". In questa sede ci serviranno solo i primi due:

- I messaggi "system" contengono le istruzioni che vanno "dette all'orecchio" del modello prima che generi la risposta. Solitamente si utilizza per le istruzioni o per informazioni aggiuntive.

- I messaggi "user" contengono la vera e propria domanda che il modello dovrà inferire, e solitamente sono l'ultimo messaggio.




In [ ]:
messaggio_system = "Sei un algoritmo cantastorie. Parla in tono fiabesco"
messaggio_user = " raccontami una storia divertente"

response = client.chat.completions.create(
    messages = [
        {"role" : "system", "content" : messaggio_system},
        {"role": "user", "content": messaggio_user}
    ],
    model = "gpt-3.5-turbo",
    temperature = 1
)

print(response.choices[0].message.content)


### Possiamo anche implementare più messaggi system, ad esempio uno con una istruzione e uno con dei dati aggiuntivi

In [ ]:
messaggio_system = "Sei un algoritmo che aiuta gli studenti. Fai ciò che ti viene chiesto"


messaggio_system2 = """
Ludovico Ariosto, uno degli scrittori più influenti del Rinascimento italiano, è noto soprattutto per il suo poema epico "Orlando Furioso". Nato il 8 settembre 1474 a Reggio Emilia, Ariosto crebbe in una famiglia benestante grazie alla posizione di suo padre come comandante della fortezza di Reggio. La famiglia si trasferì poi a Ferrara, dove Ludovico trascorse gran parte della sua vita.

La formazione di Ariosto fu dapprima legale, come desiderato dal padre, ma ben presto si orientò verso gli studi umanistici. Studiò sotto il reggente della Scuola d'Este, Gregorio da Spoleto, che gli insegnò greco e latino e gli trasmise l'amore per la letteratura classica. Durante gli anni universitari, Ariosto iniziò a scrivere poesie, influenzato dai lavori di poeti come Virgilio e Ovidio.

Dopo l'università, Ariosto entrò al servizio della corte degli Este a Ferrara, dove rimase per la maggior parte della sua vita lavorativa. Qui, iniziò la sua carriera come diplomatico e poi come capitano della fortezza di Canossa. Durante questo periodo, si dedicò anche alla scrittura e al teatro, producendo commedie che rispecchiavano lo stile e l'umorismo della commedia classica latina e delle opere di Plauto.

La sua opera più celebre, "Orlando Furioso", fu pubblicata per la prima volta nel 1516. Il poema è un ampliamento del lavoro iniziato da Matteo Maria Boiardo con "Orlando Innamorato". "Orlando Furioso" mescola elementi romantici, avventurosi e fantastici, raccontando le storie di numerosi cavalieri, dame, maghi e mostri, con un intreccio che si snoda attraverso vari continenti e sfide eroiche. La narrativa complessa e l'uso di una lingua ricca e variegata fecero di questo poema un capolavoro del Rinascimento e un modello per la letteratura epica successiva.

Ariosto rivedette "Orlando Furioso" due volte, pubblicando edizioni rinnovate nel 1521 e nel 1532, quest'ultima solo un anno prima della sua morte avvenuta il 6 luglio 1533 a Ferrara. Oltre a essere un epico poeta, Ariosto fu anche un abile amministratore e funzionario, incarichi che gli furono spesso gravosi ma che svolse con dedizione.

La vita di Ludovico Ariosto fu segnata dall'equilibrio tra le sue responsabilità alla corte degli Este e il suo impegno letterario. Nonostante le pressioni e le sfide della vita di corte, riuscì a creare opere che hanno lasciato un segno indelebile nella letteratura italiana e mondiale. La sua abilità nel tessere trame complesse, il suo uso magistrale della lingua e la sua profonda comprensione della natura umana lo rendono una figura di spicco del suo tempo.
"""


messaggio_user = "Fammi un riassunto in 50 parole"    # oppure "in quali anni rivise la sua opera?"

response = client.chat.completions.create(
    messages = [
        {"role" : "system", "content" : messaggio_system},
        {"role" : "system", "content" : messaggio_system2},
        {"role": "user", "content": messaggio_user}
    ],
    model = "gpt-3.5-turbo",
    temperature = 1
)

print(response.choices[0].message.content)